In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates
# %%

In [2]:
fldr = 'E:\\data\\dunex\\Dunex_Sig_OC\\1137sig_bar\\Exported Data\\S102410A008_outersurf\\'

In [3]:
fn = 'Burst.VTC.WAVES_20min.nc'

In [4]:
fin=fldr+fn

In [5]:
fin

'E:\\data\\dunex\\Dunex_Sig_OC\\1137sig_bar\\Exported Data\\S102410A008_outersurf\\Burst.VTC.WAVES_20min.nc'

In [6]:
WVS = xr.open_dataset(fin,group = "/Data/Waves",engine = "netcdf4")

FileNotFoundError: [Errno 2] No such file or directory: b'E:\\data\\dunex\\Dunex_Sig_OC\\1137sig_bar\\Exported Data\\S102410A008_outersurf\\Burst.VTC.WAVES_20min.nc'

In [ ]:
WVS

In [ ]:
WVS.time.attrs

### Make Stack Plot of Wave stats for time period of good data

In [ ]:
dtstart ="2021-09-14 19:01"
dtend = "2021-10-21 14:35"

In [ ]:
# define some colors using palettes from ColorBrewer https://colorbrewer2.org/
grays = ['#f7f7f7','#cccccc','#969696','#636363','#252525']
blues = ['#eff3ff','#bdd7e7','#6baed6','#3182bd','#08519c']
greens = ['#edf8e9','#bae4b3','#74c476','#31a354','#006d2c']
windcol = ['#edf8fb','#b3cde3','#8c96c6','#8856a7','#810f7c']
reds = ['#ffeda0','#feb24c','#f03b20']
divrg = ['#d7191c','#fdae61','#ffffbf','#abdda4','#2b83ba']

In [ ]:
fig, (ax) = plt.subplots(4, 1, sharex=True, figsize=(16, 10))
ax[0].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Height_Hm0, color = divrg[0],label = 'Hm0')
ax[0].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Height_Hmax, color= divrg[1], label = 'Hmax')
ax[0].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Height_H10, color= divrg[4], label = 'H10')
ax[0].set_ylabel('meters',fontsize=14)
ax[0].text(.02,.85,'Wave Heights',fontsize=16,transform=ax[0].transAxes)
ax[0].legend(loc = 'upper right')
ax[0].set_xticklabels([])
ax[0].set_title('DUNEX Pea Island: Wave stats from Signature ADCP at bar site in surf zone (1137sig)',fontsize = 18)

ax[1].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Period_Tp, color = blues[4], label = 'Tp')
ax[1].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Period_Tm02, color= blues[2], label = 'Tm')
ax[1].set_ylabel('seconds',fontsize=14)
ax[1].set_ylim((-1, 20))
ax[1].text(.02,.85,'Wave Periods',fontsize=16,transform=ax[1].transAxes)
ax[1].legend(loc = 'upper right')
ax[1].set_xticklabels([])

ax[2].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Direction_DirTp, '.',color = divrg[1], label = 'DirTp')
ax[2].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Direction_SprTp, '.',  color= blues[2], label = 'DirSprd')
ax[2].set_yticks([0.,90.,180.,270,360])
ax[2].set_ylabel('direction ($^{\circ}$T)',fontsize=14)
ax[2].text(.02,.85,'Wave direction (from)',fontsize=16,transform=ax[2].transAxes)
ax[2].legend()
ax[2].set_xticklabels([])

ax[3].plot(WVS.sel(time=slice(dtstart, dtend)).time,
         WVS.sel(time=slice(dtstart, dtend)).Pressure,color = blues[3], label = 'Pres')
ax[3].set_ylabel('decibars',fontsize=14)
ax[3].text(.02,.85,'Pressure (uncorrected)',fontsize=16,transform=ax[3].transAxes)

ax[3].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

plt.savefig('1137bar_wavs_stats_tsstack.png')

### Make Spectra plot at max(Hs)

In [ ]:
da=WVS.sel(time=slice(dtstart, dtend)).time.astype('datetime64[s]') #extract time and convert to dataframe to make string

In [ ]:
type(da)

In [ ]:
dftime=da.to_dataframe()

In [ ]:
str=dftime['time'].dt.strftime("%B %d, %Y, %r") #panda series of datetime strings

In [ ]:
type(str)

In [ ]:
#import time
#plot a spectral plot of wave energy v frequency
fig = plt.subplots(1, 1, sharex=False, figsize=(10, 8))
freq=WVS.EnergySpectra_Frequency
E=WVS.sel(time=slice(dtstart, dtend)).EnergySpectra

idx= WVS.sel(time=slice(dtstart, dtend)).Height_Hm0.argmax('time')
#idx= 1853
fwv=WVS.WaveSpectra_Frequency
E_vel=WVS.sel(time=slice(dtstart, dtend)).WaveSpectra_Vel
E_ast=WVS.sel(time=slice(dtstart, dtend)).WaveSpectra_AST
E_pres=WVS.sel(time=slice(dtstart, dtend)).WaveSpectra_Pressure


plt.plot(freq,E[:,idx], color = blues[4],linewidth = 4, label = 'Espec')
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-4, 10)
plt.ylabel('Energy, [m$^2$ $^.$ hz$^{-1}$]', fontsize = 14, fontweight = 'bold')
plt.xlabel('frequency, [hz]', fontsize = 14, fontweight = 'bold')
plt.title('Wave Spectra Plots @ max(Hs):  ' + str[int(idx)], fontsize = 16, fontweight = 'bold')

plt.plot(fwv,E_vel.isel(time=idx),'--',color = greens[3],label = 'Vel_spec')
plt.plot(fwv,E_ast.isel(time=idx),'--', color = reds[2],label = 'AST_spec')
plt.plot(fwv,E_pres.isel(time=idx),'--',color = blues[2],label = 'Pres_spec')

plt.yticks(fontsize = 12)
plt.xticks(fontsize = 12)

plt.legend()

plt.savefig('1137bar_wavs_spec_maxHs.png')

    #time.sleep(0.5)
    #break

In [ ]:
#import time
#plot a spectral plot of wave energy v frequency
fig = plt.subplots(1, 1, sharex=False, figsize=(10, 8))
freq=WVS.EnergySpectra_Frequency
E=WVS.sel(time=slice(dtstart, dtend)).EnergySpectra

idx= WVS.sel(time=slice(dtstart, dtend)).Period_Tp.argmax('time')
#idx= 1853
fwv=WVS.WaveSpectra_Frequency
E_vel=WVS.sel(time=slice(dtstart, dtend)).WaveSpectra_Vel
E_ast=WVS.sel(time=slice(dtstart, dtend)).WaveSpectra_AST
E_pres=WVS.sel(time=slice(dtstart, dtend)).WaveSpectra_Pressure


plt.plot(freq,E[:,idx], color = blues[4],linewidth = 4, label = 'Espec')
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-4, 10)
plt.ylabel('Energy, [m$^2$ $^.$ hz$^{-1}$]', fontsize = 14, fontweight = 'bold')
plt.xlabel('frequency, [hz]', fontsize = 14, fontweight = 'bold')
plt.title('Wave Spectra Plots @ max(Tp):  ' + str[int(idx)], fontsize = 16, fontweight = 'bold')

plt.plot(fwv,E_vel.isel(time=idx),'--',color = greens[3],label = 'Vel_spec')
plt.plot(fwv,E_ast.isel(time=idx),'--', color = reds[2],label = 'AST_spec')
plt.plot(fwv,E_pres.isel(time=idx),'--',color = blues[2],label = 'Pres_spec')

plt.yticks(fontsize = 12)
plt.xticks(fontsize = 12)

plt.legend()

plt.savefig('1137bar_wavs_spec_maxTp.png')


### Plot Wave Spectra time-series as pcolor/image plot

In [ ]:
import numpy as np
import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

y=freq.values
x=pd.to_datetime(dftime['time'])
X,Y=np.meshgrid(x,y)

In [ ]:
fig, ax = plt.subplots(1, 1, sharex = False, figsize = (16,4))

c = plt.pcolormesh(x, y, E, cmap='jet', shading='auto')
plt.title('Wave Frequency Spectra Time-Series @ bar site',fontsize = 16, fontweight = 'bold')
plt.xticks(rotation = 30)
plt.xticks(fontsize = 12)
plt.xticks(fontweight = 'bold')
plt.xlabel('2021', fontsize = 12, fontweight = 'bold')
plt.yticks(fontsize = 12)
plt.yticks(fontweight = 'bold')
plt.ylabel('frequency, [Hz]', fontsize = 12, fontweight = 'bold')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
fig.colorbar(c, ax=ax)

plt.savefig('1137bar_wavs_fspec_pcolor.png')


## Run-up calcs:

from Lange et al eqn [9]:

𝑅2%,𝐺 = 0.4 * 𝛽eff^0.45 ∫𝑆𝑆 𝐸^0.5 𝑓^−1.45 𝑑𝑓 + 

2 [3.72 * 𝛽eff^0.6 ∫𝑆𝑆 𝐸^0.95 𝑓^−0.25 𝑑𝑓 +0.76 * 𝛽f^2 * 𝛽eff^0.75 ∫𝑆𝑆 𝐸^0.4 𝑓^−3.1 𝑑𝑓]^0.5

term1 = 0.4 * 𝛽eff^0.45 ∫𝑆𝑆 𝐸^0.5 𝑓^−1.45 𝑑𝑓

term2 = 3.72 * 𝛽eff^0.6 ∫𝑆𝑆 𝐸^0.95 𝑓^−0.25 𝑑𝑓

term3 = 0.76 * 𝛽f^2 * 𝛽eff^0.75 ∫𝑆𝑆 𝐸^0.4 𝑓^−3.1 𝑑𝑓

where;

R2%,G = term1 + 2[term2 + term3]^0.5


In [ ]:
Ess=WVS.sel(time=slice(dtstart, dtend), EnergySpectra_Frequency=slice(0.0399,0.25)).EnergySpectra

In [ ]:
Ess.shape

In [ ]:
Hs=WVS.sel(time=slice(dtstart, dtend)).Height_Hm0

In [ ]:
Tp=WVS.sel(time=slice(dtstart, dtend)).Period_Tp

In [ ]:
time = dftime['time']

In [ ]:
f=WVS.sel(EnergySpectra_Frequency = slice(0.0399,0.25)).EnergySpectra_Frequency

In [ ]:
f

In [ ]:
dff = 0.01
beta_f = np.arange(0.1,0.01,-0.02)
beta_eff = beta_f *  (1/ 5)

In [ ]:
beta_f

In [ ]:
beta_eff

In [ ]:
R=np.empty(shape = [len(beta_f),len(time)])

In [ ]:
for j in range(0,len(beta_f)):
    R2G=[]
        
    for i in range(0,len(Hs)):

            #term1 = 0.4 * 𝛽eff^0.45 ∫𝑆𝑆 𝐸^0.5 𝑓^−1.45 𝑑𝑓
            aint = Ess[:,i]**0.5 * f**-1.45 * dff
            term1 = 0.4 * beta_eff[j]**0.45 * aint.sum()
            #term1.append(a)

            #term2 = 3.72 * 𝛽eff^0.6 ∫𝑆𝑆 𝐸^0.95 𝑓^−0.25 𝑑𝑓
            bint = Ess[:,i]**0.95 * f**-0.25 * dff
            term2= 3.72 * beta_eff[j]**0.6 * bint.sum()
            #term2.append(b)

            #term3 = 0.76 * 𝛽f^2 * 𝛽eff^0.75 ∫𝑆𝑆 𝐸^0.4 𝑓^−3.1 𝑑𝑓
            cint = Ess[:,i]**0.4 * f**-3.1 * dff
            term3= 0.76 * beta_f[j]**2 * beta_eff[j]**0.75 * cint.sum()
            #term3.append(c)

            #R2%,G = term1 + 2[term2 + term3]^0.5
            r = term1 + 2*(term2+term3)**0.5
            R2G.append(r)
            
    R[j,:]=R2G

In [ ]:
### Add Run-up calcs using Stockdon Eqn to compare:

In [ ]:
import sys
sys.path.append('c:/users/ssuttles/git/csherwood/DUNEX')
import DUNEX_funcs

In [ ]:
R2=np.empty(shape = [7,len(beta_f),len(time)])

In [ ]:
for idx,val in enumerate(beta_f):
     R2[:,idx,:]= DUNEX_funcs.calcR2(Hs,Tp,val)

In [ ]:
R2.shape

In [ ]:
dsR = xr.Dataset()
dsR['time'] = xr.DataArray(dftime['time'], dims='time')
dsR['frequency_ss']=xr.DataArray(f.values,dims='frequency_ss')
dsR['energy_ss']=xr.DataArray(Ess.values,dims=('frequency_ss','time'))
dsR['Hs']=xr.DataArray(Hs.values,dims = 'time')
dsR['Tp']=xr.DataArray(Tp.values,dims = 'time')
dsR['beta_f']=xr.DataArray(beta_f,dims='beta')
dsR['beta_eff']=xr.DataArray(beta_eff,dims='beta')
dsR['R2G'] = xr.DataArray(R, dims=('beta','time'))
dsR['R2'] = xr.DataArray(R2[0,:,:], dims=('beta','time'))

In [ ]:
dsR

In [ ]:
lstr=[]
for i in range(0,len(beta_f)):
        str = [r"$\beta_{f}$ =  %4.2f" % beta_f[i]]
        lstr = lstr + str

In [ ]:
fig, ax = plt.subplots(1, 1, sharex = False, figsize = (16,4))
plt.plot(dsR.time,dsR.R2G[0,:], label = lstr[0])
plt.plot(dsR.time,dsR.R2G[1,:], label = lstr[1])
plt.plot(dsR.time,dsR.R2G[2,:], label = lstr[2])
plt.plot(dsR.time,dsR.R2G[3,:], label = lstr[3])
plt.plot(dsR.time,dsR.R2G[4,:], label = lstr[4])
plt.title('Run-up Calculations using Lange et al Eqn 9 w/ Wave Spectral data from bar site',fontsize = 16, fontweight = 'bold')
plt.xticks(rotation = 30)
plt.xticks(fontsize = 12)
plt.xticks(fontweight = 'bold')
plt.xlabel('2021', fontsize = 12, fontweight = 'bold')
plt.yticks(fontsize = 12)
plt.yticks(fontweight = 'bold')
plt.ylabel('meters', fontsize = 12, fontweight = 'bold')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
plt.legend(fontsize = 12)
plt.savefig('PI_runup_calcs_Lange_eqn9_wavspec_bar_site.png')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex = True, figsize = (16,10))
ax[0].plot(dsR.time,dsR.R2G[0,:], label = lstr[0])
ax[0].plot(dsR.time,dsR.R2G[1,:], label = lstr[1])
ax[0].plot(dsR.time,dsR.R2G[2,:], label = lstr[2])
ax[0].plot(dsR.time,dsR.R2G[3,:], label = lstr[3])
ax[0].plot(dsR.time,dsR.R2G[4,:], label = lstr[4])
ax[0].set_title('Run-up Calculations using Lange et al Eqn 9 w/ Wave Spectral data from bar site',fontsize = 16, fontweight = 'bold')
ytks=np.arange(0,2.5,0.2)
ax[0].set_yticks(ytks)
ax[0].set_yticklabels(ytks,fontsize = 12)
ax[0].set_yticklabels(ytks,fontweight = 'bold')
ax[0].set_ylabel('meters', fontsize = 12, fontweight = 'bold')
#ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax[0].yaxis.set_major_locator(ticker.FixedLocator(ytks))
ax[0].yaxis.set_major_formatter(ticker.FormatStrFormatter("%3.1f"))
ax[0].legend(fontsize = 12)

ax[1].plot(dsR.time,dsR.R2[0,:], label = lstr[0])
ax[1].plot(dsR.time,dsR.R2[1,:], label = lstr[1])
ax[1].plot(dsR.time,dsR.R2[2,:], label = lstr[2])
ax[1].plot(dsR.time,dsR.R2[3,:], label = lstr[3])
ax[1].plot(dsR.time,dsR.R2[4,:], label = lstr[4])
ax[1].set_title('Run-up Calculations using Stockdon (2006) Eqn Wave Spectral data from bar site',fontsize = 16, fontweight = 'bold')
ax[1].set_xticklabels(ax[1].get_xticklabels(),rotation = 30)
ax[1].set_xticklabels(ax[1].get_xticklabels(),fontsize = 12)
ax[1].set_xticklabels(ax[1].get_xticklabels(),fontweight = 'bold')
ax[1].set_xlabel('2021', fontsize = 12, fontweight = 'bold')
ax[1].set_yticks(ytks)
ax[1].set_yticklabels(ytks,fontsize = 12)
ax[1].set_yticklabels(ytks,fontweight = 'bold')
ax[1].set_ylabel('meters', fontsize = 12, fontweight = 'bold')
xtks=ax[1].get_xticks()
ax[1].xaxis.set_major_locator(ticker.FixedLocator(xtks))
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax[1].yaxis.set_major_locator(ticker.FixedLocator(ytks))
ax[1].yaxis.set_major_formatter(ticker.FormatStrFormatter("%3.1f"))
#ax[1].legend(fontsize = 12)


fig.savefig('PI_runup_calcs_Lange_eqn9_wavspec_bar_site_stockdon.png')

In [ ]:
ax[1].get_xticks()

## Try hvplot:

In [ ]:
import hvplot.xarray 
import hvplot.pandas
from holoviews import opts

Need to extract into new dataset of variables with just time dim that want to plot using hvplot

In [ ]:
ds = xr.Dataset()
ds['time']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).time.astype('datetime64[s]'),dims='time')
ds['Hs']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Height_Hm0.values,dims='time')
ds['Hmax']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Height_Hmax.values,dims='time')
ds['Hmean']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Height_Hmean.values,dims='time')
ds['Tp']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Period_Tp.values,dims='time')
ds['Tm']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Period_Tm02.values,dims='time')
ds['Tmax']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Period_Tmax.values,dims='time')
ds['DirTp']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Direction_DirTp.values,dims='time')
ds['DirSprd']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Direction_SprTp.values,dims='time')
ds['Pres']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).Pressure.values,dims='time')

In [ ]:
ds.time

In [ ]:
df = ds.to_dataframe() #need to convert to pandas dataframe to plot multiple line on an axes

In [ ]:
df['time']=WVS.sel(time=slice(dtstart, dtend)).time

In [ ]:
df['dt'] = pd.to_datetime(df['time'], format='%m/%d/%Y %I:%M:%S %p')

In [ ]:
Hplot= df.hvplot(x='time', y = ['Hs', 'Hmax', 'Hmean'], width= 900, ylim = [-0.1, 5],
                 title = 'Pea Island DUNEX @ bar site: Wave Heights',ylabel = 'meters',xlabel = 'mm/dd 2021' )

In [ ]:
Hplot

In [ ]:
Tplot= df.hvplot(x='time', y = ['Tp', 'Tm'], width= 900, ylim = [-0.1,20])

In [ ]:
Tplot

In [ ]:
ds_spec=xr.Dataset() #create xarray dataset of data wavespec data from Ocean Contour 

In [ ]:
ds_spec['time']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).time,dims='time') 
#needed to round time to seconds to prevent output to netcdf being in nanoseconds
ds_spec['frequency']=xr.DataArray(WVS.EnergySpectra_Frequency.values,dims='frequency')
#ds_spec['frequency_ss']=xr.DataArray(WVS.sel(EnergySpectra_Frequency=slice(0.0399,0.25)).EnergySpectra_Frequency.values,dims='frequency_ss')
ds_spec['energy']=xr.DataArray(WVS.sel(time=slice(dtstart, dtend)).EnergySpectra.values,dims=('frequency','time'))
#ds_spec['energy_ss']=xr.DataArray(WVS.sel(time=slice(dtstart,dtend),EnergySpectra_Frequency=slice(0.0399,0.25)).EnergySpectra.values,dims=('frequency_ss','time'))

In [ ]:
ds_spec


In [ ]:
ds_spec.hvplot.image(x='time', y='frequency', z='energy',cmap='jet',width=900)

In [ ]:
ds_spec.hvplot.quadmesh(x='time', y='frequency', z='energy',cmap='jet', width=900, geo = False)

## Output results to netcdf files:

In [ ]:
## Wave spec data

In [ ]:
glob_attrs={}
glob_attrs['description']='Wave Spectral Data from DUNEX site 1137bar Nortek Signature Data - proccesed using Ocean Contour with 20 min long wave burst'
glob_attrs['source_file']=fn
glob_attrs['raw_file']='S102410A008_outersurf.ad2cp'
glob_attrs['InstrumentType:']= 'Nortek Signature 1000'
glob_attrs['start_data']=dtstart
glob_attrs['start_end']=dtend

In [ ]:
ds_spec.attrs=glob_attrs

In [ ]:
ds_spec.frequency.attrs=WVS.EnergySpectra_Frequency.attrs
ds_spec.energy.attrs=WVS.EnergySpectra.attrs


In [ ]:
fldrout = 'D:\\data\\DUNEX\\proc\\waves\\'

In [ ]:
ds_spec.to_netcdf(fldrout+'1137sig_wavespec.cdf',encoding={"time":{"dtype":"i4"}})

In [ ]:
## Wave stats data

In [ ]:
glob_attrs['description']='Wave Stats Data from DUNEX site 1137bar Nortek Signature Data - proccesed using Ocean Contour with 20 min long wave burst'

In [ ]:
ds.attrs=glob_attrs
ds.Hs.attrs=WVS.Height_Hm0.attrs
ds.Hmax.attrs=WVS.Height_Hmax.attrs
ds.Hmean.attrs=WVS.Height_Hmean.attrs
ds.Tp.attrs=WVS.Period_Tp.attrs
ds.Tm.attrs=WVS.Period_Tm02.attrs
ds.Tmax.attrs=WVS.Period_Tmax.attrs
ds.DirTp.attrs=WVS.Direction_DirTp.attrs
ds.DirSprd.attrs=WVS.Direction_SprTp.attrs
ds.Pres.attrs=WVS.Pressure.attrs

In [ ]:
ds.to_netcdf(fldrout+'1137sig_wavestats.cdf',encoding={"time":{"dtype":"i4"}})

In [ ]:
## Run-up calcs data and results

In [ ]:
glob_attrs['description']='Runup Calculations and input data from DUNEX Pea Island site 1137bar Nortek Signature Data: using Lange et al Eqn 9 & Stockdon Eqn'

In [ ]:
dsR.attrs=glob_attrs
dsR.frequency_ss.attrs=WVS.EnergySpectra_Frequency.attrs
dsR.frequency_ss.attrs['description']='Sea/swell frequency band used in runup calcs'
dsR.energy_ss.attrs=WVS.EnergySpectra.attrs
dsR.energy_ss.attrs['description']='Sea/swell enegry spectra used in used in run-up calcs'
dsR.Hs.attrs=WVS.Height_Hm0.attrs
dsR.Tp.attrs=WVS.Period_Tp.attrs
dsR.beta_f.attrs={'description':'foreshore beach slope parameter used in runup equations','units':'none'}
dsR.beta_eff.attrs={'description':'effective mid-surfzone slope parameter used in runup equations','units':'none','note':'for these calcs beta_eff was estimated as beta_f*1/5'}
dsR.R2G.attrs = {'description':'runup estimates using Eqn 9 from Lange et al','units':'meters'}
dsR.R2.attrs = {'description':'runup estimates using Stockdon Eqn','units':'meters'}

In [ ]:
dsR.to_netcdf(fldrout+'1137sig_runup_calcs.cdf',encoding={"time":{"dtype":"i4"}})